<a href="https://colab.research.google.com/github/atlanticsharks/atlanticsharks.github.io/blob/main/MAYABAY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. SET UP

https://github.com/filippovarini/sharktrack/blob/master/sharktrack-user-guide.md

Mount google drive files to /content/drive so that they can be accessed within this script


In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Clone the git repository for sharktrack AI originally from github.com/filippovarini but modified for google collab here

In [ ]:
!git clone https://github.com/atlanticsharks/sharktrackAI.git sharktrack

Cloning into 'sharktrack'...
remote: Enumerating objects: 1297, done.
remote: Counting objects: 100% (197/197), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 1297 (delta 120), reused 128 (delta 71), pack-reused 1100 (from 1)
Receiving objects: 100% (1297/1297), 320.64 MiB | 28.37 MiB/s, done.
Resolving deltas: 100% (647/647), done.


Check that the git respository has been correctly linked


In [ ]:
!ls /content/sharktrack

annotation-pipelines.md  input_videos	models	   requirements.txt	     static  trackers
app.py			 MAYABAY.ipynb	readme.md  sharktrack-user-guide.md  tests   utils


set as working directory:

In [ ]:
import os
os.chdir('/content/sharktrack')

Now, let's set up our environment and install requirements:

In [ ]:
!apt install python3-venv

In [ ]:
!python3 -m venv venv
!source venv/bin/activate
!pip install -r requirements.txt
!pip install pandas==2.2.2

# 2. Run Sharktrack AI


Now what we need to do, is to be able to run 'app.py' from 'sharktrack' but while bringing the 'input_videos' path to /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/input_videos

In [ ]:
!python /content/sharktrack/app.py \
--input /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/input_videos/ \
--output /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/outputs \
--resume # if the sharktrack AI stopped early

## 2.2 When the AI finishes...

make a sound when done

In [ ]:
import os
# For macOS
os.system("afplay /System/Library/Sounds/Tink.aiff")

unassign the runtime when done so as to not go over limits

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
!ls /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/input_videos/

# 2.3. **Now, go and annotate the videos by hand before you return for the next steps.**


# 3. COMPUTE MaxN


## 3.0 Before computing MaxN must do this *one* time.
The video path column in outputs.csv is blank. We can add it here by appending the path with the video name like this:

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/outputs/internal_results/output.csv')
print(df.head()) # check if needed


                                          video_path  \
0  /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/...   
1  /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/...   
2  /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/...   
3  /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/...   
4  /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/...   

                   video_name   frame               time         xmin  \
0  B_MY2_01_181223_1700E1.MP4   541.0  00h:00m:09s:009ms    19.775038   
1  B_MY2_01_181223_1700E1.MP4   561.0  00h:00m:09s:342ms     3.485797   
2  B_MY2_01_181223_1700E1.MP4   581.0  00h:00m:09s:676ms     0.280212   
3  B_MY2_01_181223_1700E1.MP4  1261.0  00h:00m:21s:021ms  1218.452026   
4  B_MY2_01_181223_1700E1.MP4  1881.0  00h:00m:31s:364ms  1949.991333   

          ymin         xmax         ymax       w       h  confidence  \
0    30.410229  2650.185059   795.674011  2704.0  1520.0    0.492467   
1    34.629547  2675.065430  1060.798706  2704.0  1520.0    0.446016   
2    33.

<ipython-input-45-0d2c2f244775>:3: DtypeWarning: Columns (0,1,3,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/outputs/internal_results/output.csv')


In [ ]:
# append video_name to video_path
video_path = '/content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/input_videos/'
df['video_path'] = video_path + df['video_name']

original_file_path = '/content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/outputs/internal_results/output.csv'
df.to_csv(original_file_path, index=False, mode='w')

Check if it worked by printing the first few rows of data

In [ ]:
print(df.tail())

                                               video_path  \
621990  /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/...   
621991  /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/...   
621992  /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/...   
621993  /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/...   
621994  /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/...   

                        video_name    frame               time        xmin  \
621990  B_MY2_10_201223_1855E3.MP4  25821.0  00h:07m:10s:763ms  627.398010   
621991  B_MY2_10_201223_1855E3.MP4  25861.0  00h:07m:11s:431ms  403.202881   
621992  B_MY2_10_201223_1855E3.MP4  25881.0  00h:07m:11s:764ms  151.268478   
621993  B_MY2_10_201223_1855E3.MP4  25901.0  00h:07m:12s:098ms   47.332443   
621994  B_MY2_10_201223_1855E3.MP4  25921.0  00h:07m:12s:432ms    0.384940   

              ymin         xmax         ymax       w       h  confidence  \
621990  491.133453  1145.000488   884.326050  1920.0  1080.0    0.736510   
621991  652.

In [ ]:
print(df.head())

                                          video_path  \
0  /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/...   
1  /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/...   
2  /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/...   
3  /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/...   
4  /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/...   

                   video_name   frame               time         xmin  \
0  B_MY2_01_181223_1700E1.MP4   541.0  00h:00m:09s:009ms    19.775038   
1  B_MY2_01_181223_1700E1.MP4   561.0  00h:00m:09s:342ms     3.485797   
2  B_MY2_01_181223_1700E1.MP4   581.0  00h:00m:09s:676ms     0.280212   
3  B_MY2_01_181223_1700E1.MP4  1261.0  00h:00m:21s:021ms  1218.452026   
4  B_MY2_01_181223_1700E1.MP4  1881.0  00h:00m:31s:364ms  1949.991333   

          ymin         xmax         ymax       w       h  confidence  \
0    30.410229  2650.185059   795.674011  2704.0  1520.0    0.492467   
1    34.629547  2675.065430  1060.798706  2704.0  1520.0    0.446016   
2    33.

## 3.1.  Compute MaxN (*finally!*)

You can compute MaxN locally by following the steps below:

Open the Prompt/Terminal at the sharktrack folder location as you did when running the model in the User Guide step.

Activate the virtual environment (guide here)

Run python utils/compute_maxn.py --path <output_path>, replacing <output_path> with the path of the folder that contains the output.csv file and all the detection folders.

Example: python utils/compute_maxn.py --path output

You can optionally provide a path to the original videos, to output the MaxN jpg Frames

You will see a maxn.csv file in the SharkTrack folder under outputs / analysed

The MaxN jpg frames get saved to the input_videos folder (because of a weird issue with google collab) in seperate folders for each video.


In [ ]:
!python utils/compute_maxn.py --path /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/outputs


Path to original videos (to compute maxn screenshots) [N/A]: /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/input_videos
Are your videos split in chapters? [y/N]: N
Computing MaxN from annotations cleaned locally...
/content/sharktrack/utils/compute_maxn.py:46: DtypeWarning: Columns (0,1,3,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(original_output_path)
/content/sharktrack/utils/compute_maxn.py:66: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  maxn = frame_box_cnt.groupby(["video_path", "video_name", "label"], as_index=False, dropna=False).apply(lambda grp: grp.nlargest(1, "n"))
MaxN computed! Check in the folder /content/drive/MyDrive/3.2

In [ ]:
 #input video path:

#  /content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/input_videos

### Checking some things. Not needed.


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/outputs/internal_results/overview.csv')
print(df.tail()) # check if needed

In [ ]:
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('/content/drive/MyDrive/3.2_MAYA_BAY_BLACKTIPS/outputs/internal_results/output.csv')

# Filter the DataFrame for rows where the video name is 'B_MY2_20_231223_1145E1.MP4'
filtered_df = df[df['video_name'] == 'B_MY2_10_201223_1855E3.MP4']

# Print the filtered DataFrame
print(filtered_df)

## *how to make google collab not disconect when running the code for a long time*

As of June 2023, here's the working javascript code for preventing Google colab from disconnecting due to inactivity.

```
 function keepAliveProgrammatically() {
  document.querySelector('colab-connect-button').shadowRoot.querySelector("#connect").click();
}
```
The function above is responsible for clicking the compute resources button It is clicked after every 60 seconds. The code snippet below is responsible for running the function.
```
const keepAliveProgrammaticallyInterval = setInterval(() => {
    keepAliveProgrammatically();
}, 60000);
```
To stop the code, use the code snippet below.
```
clearInterval(keepAliveProgrammaticallyInterval);
```